In [ ]:
from __future__ import print_function
import os.path
import pandas as pd
import sys
sys.path.insert(0, '../..')
%load_ext autoreload
from JKBio import Helper as h
%autoreload 2
%load_ext rpy2.ipython
import numpy as np
from bokeh.plotting import *
from bokeh.models import HoverTool
output_notebook()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from JKBio.helper import pyDESeq2 as pyDESeq2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
#from umap import UMAP

# we first need to recreate out RNA matrix from slamdunk's tcount files

In [ ]:
ls ../data/slamseq_data

In [ ]:
location= '../data/slamseq_data/*.tsv' # ../data/slamseq_data/

In [ ]:
files = ! ls $location
files

In [ ]:
files = files[:6] +[files[i] for i in [-8,-6,-4]] + files[-3:]

In [ ]:
data={}
for file in files:
    data[file.split('/')[-1].split('.')[0]]= pd.read_csv(file, sep='\t', comment='#',header=0)

In [ ]:
files

In [ ]:
prev = -2
for k,val in data.items():
    if len(set(val.Name)) != prev and prev!=-2: 
        raise ValueError('we do not have the same number of genes in each file')
    prev = len(set(val.Name))

![](images/conversionrates.png)

In [ ]:
data.keys()

In [ ]:
val

In [ ]:
flag_var= 100

readcounts = {i:[0]*len(data) for i in val.Name.unique()}
tccounts = {i:[0]*len(data) for i in val.Name.unique()}

In [ ]:
for n, (_,val) in enumerate(data.items()):
    val = val.sort_values(by="Name")
    j  = 0
    #print('              ',end='\r')
    readcount = [val.iloc[0].ReadCount]
    tccount = [val.iloc[0].TcReadCount]
    prevname = val.iloc[0].Name
    for _, v in val.iloc[1:].iterrows():
        if v.Name==4609:
            print(v.ReadCount,v.TcReadCount)
            print(readcount,tccount)
        if v.Name == prevname:
            readcount.append(v.ReadCount)
            tccount.append(v.TcReadCount)
        else:
            readcounts[prevname][n] = np.sum(readcount)
            tccounts[prevname][n] = np.sum(tccount)
            #if np.var(readcount) > flag_var:
            #    print("pb with "+str(v.Name))
            prevname=v.Name
            j+=1
            #print(j,end='\r')
            readcount = [v.ReadCount]
            tccount = [v.TcReadCount]
            
files = [*data]

In [ ]:
val.Name

In [ ]:
files

In [ ]:
readcounts[4609]

In [ ]:
readcounts = pd.DataFrame(data=readcounts,columns=val.Name.unique(),index=data.keys()).T
tccounts = pd.DataFrame(data=tccounts,columns=val.Name.unique(),index=data.keys()).T

In [ ]:
readcounts.loc[4609]

In [ ]:
names, _ = h.convertGenes(readcounts.index.tolist(), from_idtype="entrez_id", to_idtype="symbol")
readcounts.index = names

In [ ]:
names, _ = h.convertGenes(tccounts.index.tolist(), from_idtype="entrez_id", to_idtype="symbol")
tccounts.index = names

In [ ]:
todrop=np.argwhere(tccounts.values.max(1)>0).ravel()
tccounts = tccounts.iloc[todrop]
readcounts = readcounts.iloc[todrop]

In [ ]:
todrop=np.argwhere(readcounts.values[:,:-1].max(1)>=10).ravel()
tccounts = tccounts.iloc[todrop]
readcounts = readcounts.iloc[todrop]

In [ ]:
tccounts

## SLAM-seq
All SLAM-seq assays were performed at 60-70% confluency for adherent cells or
60% of the maximum cell density counted on a hemocytometer for suspension cells. 5-7h
prior to each assay, growth medium was aspirated and replaced. Unless stated otherwise,
cells were pre-treated with indicated small molecule inhibitors or 100µM IAA for 30 min
to pre-establish full target inhibition or degradation. Newly synthesized RNA was labeled
for indicated time spans (45 min or 60 min) at a final concentration of 100µM 4-
thiouridine (4sU, Carbosynth). Adherent cells were harvested by direct snap-freezing of
plates on dry ice. Suspension cells were spun down and immediately snap-frozen. RNA
extraction was performed using the RNeasy Plus Mini Kit (Qiagen). 

Total RNA was subjected to alkylation by iodoacetamide (Sigma, 10mM) for 15 min and RNA was repurified by ethanol precipitation. 500ng alkylated RNA were used as input for generating
3’-end mRNA sequencing libraries using a commercially available kit (QuantSeq 3′
mRNA-Seq Library Prep Kit FWD for Illumina and PCR Add-on Kit for Illumina,
Lexogen). Deep sequencing was performed using HiSeq1500 and HiSeq2500 platforms
(Illumina).

In [ ]:
tccounts

# lets look at the similarity accross replicates

we have some outliers, else it seems to make some sense and on average, to be extremelly similar!

In [ ]:
tccounts.columns

In [ ]:
%matplotlib inline
sns.heatmap(tccounts.corr(), 
            xticklabels=tccounts.columns.values,
            yticklabels=tccounts.columns.values)

In [ ]:
readcounts = readcounts[readcounts.columns[sorting]]

In [ ]:
%matplotlib inline
sns.heatmap(readcounts.corr(), 
            xticklabels=readcounts.columns.values,
            yticklabels=readcounts.columns.values)

read the paper for MV411 QC steps.

### A quick detour to look for clustering

## Getting the Core TF information

In [ ]:
ctf=pd.read_csv('../data/CTF.csv',header=None)[0].values.tolist()
ctf

In [ ]:
set(ctf) & set(tccounts.index)

In [ ]:
ctfpos = [val for val in tccounts.index if val in ctf]
notctfpos = [val for val in tccounts.index if val not in ctf]

In [ ]:
ctfpos.extend(['EGR1','SERTAD1'])

We find a CTF not in the dataset

# Let's look at how they change accross time

how core transcription factors change accross time when the cell is with JQ1 conditions

we are looking at the difference between production of each CTF compared to the average RNA production with JQ1 and without JQ1

## DESEQ ANALYSIS

In [ ]:
readcounts["genes"] = readcounts.index
readcounts = readcounts.reset_index(drop=True)
tccounts["genes"] = tccounts.index
tccounts = tccounts.reset_index(drop=True)

In [ ]:
tccounts.iloc[140]

In [ ]:
tccounts.columns

In [ ]:
designK562 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','JQ1'], 
                      data=np.array([[1,1,1,0,0,0,0,0,0,0,0,0],
                                     [0,0,0,1,1,1,0,0,0,0,0,0]],dtype=bool).T)
designMV411 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','JQ1'], 
                      data=np.array([[0,0,0,0,0,0,1,1,1,0,0,0],
                                     [0,0,0,0,0,0,0,0,0,1,1,1]],dtype=bool).T)

### Differential gene expression analysis, PCA and GO-term enrichment

For gene-level analysis, raw reads mapped to different UTR annotations of the same gene were summed up by Entrez Gene ID. Pilot studies of K562 cells with kinase inhibitors were performed as single experiments. 

Analysis of differential gene expression was restricted to genes with ≥ 10 reads in at least one condition for 50bp sequencing runs (flavopiridol and DMSO) or ≥ 20 reads in at least one condition for 100bp sequencing runs (mk2206, trametinib, nilotinib, trametinib + mk2206 and DMSO). For estimating differential expression, a pseudo-count of 1 raw read was added to all genes.

#### All other SLAM-seq experiments were performed in triplicates and analyzed as follows. 
Differential gene expression calling was performed on raw read counts with ≥ 2 T>C conversions using DESeq2 (version 1.14.1) with default settings, and with size factors estimated on corresponding total mRNA reads for global normalization.

Downstream analysis was restricted to genes passing all internal filters for FDR estimation by DESeq2. Principal component analysis was performed after variance
stabilizing transformation on the 500 most variable genes across all conditions of a given
experiment. GO-term enrichment analysis was performed on genes significantly and strongly downregulated (FDR ≤ 0.1, log2FC ≤ -1) in SLAM-seq upon IAA-treatment in K562MYC-AID + Tir1 by the PANTHER Overrepresentation Test (Fisher's Exact with FDR multiple test correction, release 20171205, http://pantherdb.org) on GO Ontology database Released 2017-12-27.

In [ ]:
tccounts.columns = [i.replace('-','.') for i in tccounts.columns]

In [ ]:
tccounts[tccounts.genes=='MYC']

In [ ]:
deseqK562 = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array([1,1,1,1,1,1,0,0,0,0,0,0,1],np.bool)]],
                         design_matrix=designK562[np.array([1,1,1,1,1,1,0,0,0,0,0,0],np.bool)],
                         design_formula="~DMSO - JQ1",
                         gene_column="genes")
deseqMV411 = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array([0,0,0,0,0,0,1,1,1,1,1,1,1],np.bool)]],
                         design_matrix=designMV411[np.array([0,0,0,0,0,0,1,1,1,1,1,1],np.bool)],
                         design_formula="~DMSO - JQ1",
                         gene_column="genes")

In [ ]:
deseqK562.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[np.array([1,1,1,0,0,0,0,0,0,0,0,0,0],np.bool)]].values+1), 1)))
deseqMV411.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[np.array([0,0,0,0,0,0,1,1,1,0,0,0,0],np.bool)]].values+1), 1)))

In [ ]:
deseqMV411.run_deseq()
deseqK562.run_deseq()

In [ ]:
deseqMV411.get_deseq_result()
deseqK562.get_deseq_result()

In [ ]:
MV411res = deseqMV411.deseq_result
K562res = deseqK562.deseq_result

In [ ]:
MV411res.pvalue = np.nan_to_num(np.array(MV411res.pvalue), 1)
MV411res.log2FoldChange = np.nan_to_num(np.array(MV411res.log2FoldChange), 0)
K562res.pvalue = np.nan_to_num(np.array(K562res.pvalue), 1)
K562res.log2FoldChange = np.nan_to_num(np.array(K562res.log2FoldChange), 0)

In [ ]:
MV411res.log2FoldChange = -MV411res.log2FoldChange
K562res.log2FoldChange = -K562res.log2FoldChange
MV411res["gene_id"] = MV411res.genes
K562res["gene_id"] = K562res.genes

In [ ]:
len(np.where(tccounts.max(1)>70)[0])

In [ ]:
ntodrop = np.where(tccounts.max(1)>30)[0]
MV411res = MV411res.iloc[ntodrop]
K562res = K562res.iloc[ntodrop]

In [ ]:
mix = pd.DataFrame()

In [ ]:
mix["gene_id"] = K562res["gene_id"]

In [ ]:
mix['x'] = - MV411res.log2FoldChange
mix['y'] = K562res.log2FoldChange

In [ ]:
h.scatter(mix[['x','y']].values[:12000], 
               mix['gene_id'].values.tolist()[:12000], radi= 0.06, alpha=0.3,
              colors = [0 if i in ctfpos else 1 for i in mix['gene_id'].values.tolist()[:12000]])

we can conclude that we get similar results to the slamseq myc paper although it seems that our values are a bit skewed toward higher expression than what is on the slamseq paper. It mightt be explained by the pseudo count of 1 that I did not set. Because I think it would highly bias the DESeq algorithm.

In [ ]:
mix[['x','y']].values[:100]

In [ ]:
MV411res["gene_id"]